# Quantum Phase Estimation (QPE) - Precision Measurement with Quantum Speedup

Quantum Phase Estimation is a fundamental quantum algorithm that enables precise estimation of eigenphases of unitary operators. It serves as the foundation for many important quantum algorithms including Shor's factoring algorithm and quantum simulation techniques.

In [ ]:
# Import required libraries
import time

from ariadne import explain_routing, simulate
from ariadne.algorithms import AlgorithmParameters, QuantumPhaseEstimation

## Understanding Quantum Phase Estimation

QPE finds the eigenphase φ of a unitary operator U with eigenstate |ψ⟩:

$U|\psi\rangle = e^{2\pi i\phi}|\psi\rangle$

**Classical complexity:** O(1/ε) for precision ε
**Quantum complexity:** O(log(1/ε)) for precision ε

Key components:
1. **Estimation register**: n qubits for phase representation
2. **Eigenstate register**: Contains the eigenstate |ψ⟩
3. **Controlled-U operations**: Apply powers of U to create phase kickback
4. **Inverse QFT**: Extract binary representation of phase

## Creating QPE Circuits with Different Precision

In [ ]:
# Create QPE circuits with different precision levels
# Total qubits = estimation qubits + 1 eigenstate qubit
qpe_configs = [
    {"total_qubits": 3, "name": "QPE-2 (1-bit precision)"},
    {"total_qubits": 4, "name": "QPE-3 (2-bit precision)"},
    {"total_qubits": 5, "name": "QPE-4 (3-bit precision)"},
    {"total_qubits": 6, "name": "QPE-5 (4-bit precision)"},
]

qpe_circuits = {}

for config in qpe_configs:
    n_qubits = config["total_qubits"]
    params = AlgorithmParameters(n_qubits=n_qubits)

    qpe = QuantumPhaseEstimation(params)
    circuit = qpe.create_circuit()
    qpe_circuits[n_qubits] = circuit

    n_estimation = n_qubits - 1
    precision_bits = n_estimation

    print(f"{config['name']}:")
    print(f"  Total qubits: {n_qubits}")
    print(f"  Estimation qubits: {n_estimation}")
    print(f"  Precision: {precision_bits} bits")
    print(f"  Circuit depth: {circuit.depth()}")
    print(f"  Gate counts: {circuit.count_ops()}")
    print()

## Visualizing QPE Circuit Structure

In [ ]:
# Display the 4-qubit QPE circuit (3 estimation qubits)
print("4-qubit QPE Circuit (3 estimation qubits, 1 eigenstate qubit):")
print(qpe_circuits[4].draw(output="text"))

## Testing QPE Across Different Backends

In [ ]:
# Test 4-qubit QPE across different backends
test_circuit = qpe_circuits[4]
backends = ["stim", "qiskit", "mps", "tensor_network"]
results = {}

print("Testing QPE-4 across backends:")
print("=" * 50)

for backend in backends:
    try:
        start_time = time.time()
        result = simulate(test_circuit, shots=1000, backend=backend)
        end_time = time.time()

        results[backend] = {
            "time": end_time - start_time,
            "counts": result.counts,
            "backend_used": result.backend_used.value,
        }

        print(f"{backend:15} | {end_time - start_time:8.4f}s | {result.backend_used.value}")

    except Exception as e:
        print(f"{backend:15} | FAILED - {str(e)[:30]}")
        results[backend] = {"error": str(e)}

## Analyzing Phase Estimation Results

In [ ]:
# Analyze the phase estimation results
# For our implementation, we expect to estimate phase φ = 0.5 (eigenvalue -1 of Z gate)
expected_phase = 0.5
expected_binary = "10"  # 0.5 in 2-bit binary

print("\nQPE Results Analysis:")
print("=" * 40)
print(f"Expected phase: {expected_phase}")
print(f"Expected binary (2 bits): {expected_binary}")
print()

for backend, data in results.items():
    if "error" not in data:
        counts = data["counts"]
        total_shots = sum(counts.values())

        print(f"\n{backend.upper()} Results:")

        # Show all measurement outcomes
        sorted_counts = sorted(counts.items(), key=lambda x: x[0])

        for outcome, count in sorted_counts:
            probability = count / total_shots

            # Extract estimation bits (first 3 bits for 4-qubit case)
            if len(outcome) >= 3:
                estimation_bits = outcome[:3]
                eigenstate_bit = outcome[3] if len(outcome) > 3 else "0"

                # Convert estimation bits to phase
                estimated_phase = int(estimation_bits, 2) / 2 ** len(estimation_bits)

                print(f"  |{outcome}⟩: {count:4d} ({probability:.3f}) → phase ≈ {estimated_phase:.3f}")
            else:
                print(f"  |{outcome}⟩: {count:4d} ({probability:.3f})")

## Precision Analysis: How Accuracy Improves with More Qubits

In [ ]:
# Test how precision improves with more estimation qubits
precision_results = {}
test_backend = "qiskit"  # Use reliable backend for precision test

print(f"QPE Precision Analysis (backend: {test_backend}):")
print("=" * 60)
print(f"{'Total Qubits':<12} | {'Estimation':<11} | {'Precision':<9} | {'Time (s)':<9} | {'Most Likely':<12}")
print("-" * 60)

for n_qubits in [3, 4, 5, 6]:
    if n_qubits in qpe_circuits:
        circuit = qpe_circuits[n_qubits]
        n_estimation = n_qubits - 1

        try:
            start_time = time.time()
            result = simulate(circuit, shots=1000, backend=test_backend)
            end_time = time.time()

            # Find most likely outcome
            counts = result.counts
            most_likely_outcome = max(counts.items(), key=lambda x: x[1])[0]

            # Extract estimation bits and convert to phase
            if len(most_likely_outcome) >= n_estimation:
                estimation_bits = most_likely_outcome[:n_estimation]
                estimated_phase = int(estimation_bits, 2) / 2**n_estimation

                precision = 1 / 2**n_estimation

                print(
                    f"{n_qubits:<12} | {n_estimation:<11} | {precision:<9.3f} | {end_time - start_time:<9.4f} | {estimated_phase:<12.3f}"
                )

                precision_results[n_qubits] = {
                    "time": end_time - start_time,
                    "precision": precision,
                    "estimated_phase": estimated_phase,
                    "most_likely": most_likely_outcome,
                }

        except Exception as e:
            print(f"{n_qubits:<12} | {n_estimation:<11} | FAILED - {str(e)[:20]}")

## QPE Circuit Complexity Analysis

In [ ]:
# Analyze circuit properties for different QPE configurations
print("\nQPE Circuit Properties:")
print("=" * 30)

for n_qubits in sorted(qpe_circuits.keys()):
    params = AlgorithmParameters(n_qubits=n_qubits)
    qpe = QuantumPhaseEstimation(params)
    analysis = qpe.analyze_circuit_properties()

    n_estimation = n_qubits - 1

    print(f"\nQPE-{n_qubits} ({n_estimation} estimation qubits):")
    print(f"  Total qubits: {analysis['n_qubits']}")
    print(f"  Depth: {analysis['depth']}")
    print(f"  Total gates: {analysis['size']}")
    print(f"  Two-qubit gates: {analysis['two_qubit_gates']}")
    print(f"  Entanglement heuristic: {analysis['entanglement_heuristic']:.2f}")
    print(f"  Gate types: {list(analysis['gate_counts'].keys())}")

    # Calculate theoretical precision
    precision = 1 / 2**n_estimation
    print(f"  Theoretical precision: ±{precision:.3f}")

## Understanding Controlled-U Operations

In [ ]:
# Demonstrate the controlled-U operations in QPE
def analyze_controlled_u_operations(n_qubits):
    """Analyze the controlled-U operations in QPE circuit."""
    params = AlgorithmParameters(n_qubits=n_qubits)
    qpe = QuantumPhaseEstimation(params)
    circuit = qpe.create_circuit()

    n_estimation = n_qubits - 1

    print(f"Controlled-U Operations for QPE-{n_qubits}:")
    print(f"Estimation qubits: {n_estimation}")
    print(f"Eigenstate qubit: {n_qubits - 1}")
    print()

    # Count controlled operations
    controlled_operations = 0
    for instruction, qubits, _ in circuit.data:
        if instruction.name in ["cp", "cu", "cx"] and len(qubits) > 1:
            controlled_operations += 1

    print(f"Total controlled operations: {controlled_operations}")
    print(f"Theoretical controlled-U^(2^k) operations: {n_estimation}")
    print()

    # Show the structure of controlled operations
    print("Controlled operation pattern:")
    for i in range(n_estimation):
        repetitions = 2**i
        print(f"  Estimation qubit {i} → Eigenstate: U^{repetitions}")


analyze_controlled_u_operations(4)

## Educational Content: QPE Mathematical Background

In [ ]:
# Get educational content about QPE
params = AlgorithmParameters(n_qubits=4)
qpe = QuantumPhaseEstimation(params)
educational_content = qpe.get_educational_content()

print("=== QPE Mathematical Background ===")
print(educational_content["mathematical_background"])

print("\n=== Implementation Notes ===")
print(educational_content["implementation_notes"])

print("\n=== Applications ===")
print(educational_content["applications"])

## Routing Analysis for QPE

In [ ]:
# Analyze how Ariadne routes QPE circuits
print("Routing Analysis for QPE-4:")
print("=" * 30)

explanation = explain_routing(test_circuit)
print(explanation)

## Key Takeaways

1. **Exponential Precision**: QPE achieves exponential precision improvement with linear qubit increase
2. **Phase Kickback**: Controlled-U operations transfer phase information to estimation qubits
3. **Inverse QFT**: Essential for extracting binary representation of the phase
4. **Circuit Complexity**: Depth scales as O(n²) due to controlled-U operations and inverse QFT
5. **Practical Applications**: Foundation for Shor's algorithm, quantum chemistry, and simulation

Quantum Phase Estimation demonstrates how quantum algorithms can achieve precision that would require exponentially more resources classically, making it a cornerstone of quantum computing for scientific and computational applications.